In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
model = ChatOpenAI()

In [4]:
class BatsmanState(TypedDict):
    runs: int
    balls: int
    fours: int
    sixes: int
    sr: float
    bpb: float
    boundary_percentage: float
    summary: str

In [5]:
def calculate_sr(state: BatsmanState):
    runs = state['runs']
    balls = state['balls']
    sr = runs/balls*100
    return {'sr': sr}


In [6]:
def calculate_bpb(state: BatsmanState):
    balls = state['balls']
    fours = state['fours']
    sixes = state['sixes']
    bpb = balls/(fours+sixes)
    return {'bpb': bpb}

In [7]:
def calculate_boundary_percentage(state: BatsmanState):
    runs = state['runs']
    boundary_percentage = (((state['fours']*4)+(state['sixes']*6))/runs)*100
    return {'boundary_percentage': boundary_percentage}

In [13]:
def summary(state: BatsmanState):

    summary = f"""
Strike Rate - {state['sr']} \n
Balls per boundary - {state['bpb']} \n
Boundary percent - {state['boundary_percentage']}
"""
    
    return {'summary': summary}

In [14]:
graph = StateGraph(BatsmanState)
graph.add_node('calculate_sr', calculate_sr)
graph.add_node('calculate_bpb', calculate_bpb)
graph.add_node('calculate_boundary_percentage', calculate_boundary_percentage)
graph.add_node('summary', summary)

#edges
graph.add_edge(START, 'calculate_sr')
graph.add_edge(START, 'calculate_bpb')
graph.add_edge(START, 'calculate_boundary_percentage')

graph.add_edge('calculate_sr', 'summary')
graph.add_edge('calculate_bpb', 'summary')
graph.add_edge('calculate_boundary_percentage', 'summary')

graph.add_edge('summary', END)

workflow = graph.compile()

In [15]:
initial_state = {'runs': 100,
                 'balls': 50,
                 'fours': 6,
                 'sixes': 7
}
final_state = workflow.invoke(initial_state)
print(final_state['summary'])


Strike Rate - 200.0 

Balls per boundary - 3.8461538461538463 

Boundary percent - 66.0

